In [ ]:
!pip install azure-ai-textanalytics

In [ ]:
import os
import pandas as pd
from getpass import getpass
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

In [ ]:
API_KEY = getpass("Insira sua chave da API do Copilot:")

In [ ]:
ENDPOINT = "https://boamentelanguageservice.cognitiveservices.azure.com/"

In [ ]:
def autenticar_cliente():
    credential = AzureKeyCredential(API_KEY)
    return TextAnalyticsClient(endpoint=ENDPOINT, credential=credential)

cliente = autenticar_cliente()

In [ ]:
# Função para analisar sentimentos e detectar ideação suicida
def analisar_sentimento(frase):
    try:
        response = cliente.analyze_sentiment([frase])[0]
        sentimento = response.sentiment

        # Regra simples: se for "negative" e tiver alta confiança, pode indicar risco
        if sentimento == "negative" and response.confidence_scores.negative > 0.8:
            return "1"  
        return "0" 
    except Exception as e:
        return f"Erro: {e}"

In [ ]:
def processar_csv(pasta_entrada, pasta_saida):
    for i in range(1, 11):
        entrada = f"{pasta_entrada}/dataset_part{i}.csv"
        saida = f"{pasta_saida}/azure_dataset_part{i}.csv"

        if os.path.exists(entrada):
            df = pd.read_csv(entrada)
            if "text" in df.columns:
                df["AzureSentiment"] = df["text"].apply(analisar_sentimento)
                df.to_csv(saida, index=False)
                print(f"Salvo: {saida}")
            else:
                print(f"Erro: coluna 'text' não encontrada em {entrada}")
        else:
            print(f"Arquivo não encontrado: {entrada}")

print("Processamento concluído!")

In [ ]:
pasta_entrada = "./DATA-SPLIT"
pasta_saida = "./AI-Azure"

os.makedirs(pasta_entrada, exist_ok=True)
os.makedirs(pasta_saida, exist_ok=True)

In [ ]:
processar_csv(pasta_entrada, pasta_saida)